In [1]:
## In sagemaker studio
!pip install sagemaker-studio-image-build

  Using cached sagemaker_studio_image_build-0.6.0.tar.gz (13 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for sagemaker-studio-image-build: filename=sagemaker_studio_image_build-0.6.0-py3-none-any.whl size=13468 sha256=37baddf98e77429ac5b3cad437d554f3e28e7a6121bd8fb0aaa4ef80790a672f
  Stored in directory: /home/ec2-user/.cache/pip/wheels/69/7b/d1/1318b1530ee5322c9be00f206badea11b5148626ef58c0e0dc
Successfully built sagemaker-studio-image-build


In [3]:
pip install pandas==1.5.3


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 72.0 MB/s eta 0:00:0000:010:01
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dask-expr 1.1.9 requires pandas>=2, but you have pandas 1.5.3 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install --upgrade setuptools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 18.2 MB/s eta 0:00:0000:010:01
  Attempting uninstall: setuptools
    Found existing installation: setuptools 71.0.4
    Uninstalling setuptools-71.0.4:
      Successfully uninstalled setuptools-71.0.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sparkmagic 0.21.0 requires pandas<2.0.0,>=0.17.1, but you have pandas 2.2.2 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install setuptools==58.0.4


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 816.5/816.5 kB 10.1 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: setuptools
    Found existing installation: setuptools 74.0.0
    Uninstalling setuptools-74.0.0:
      Successfully uninstalled setuptools-74.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sparkmagic 0.21.0 requires pandas<2.0.0,>=0.17.1, but you have pandas 2.2.2 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [2]:
!sh build_and_push_studio.sh

build_and_push_studio.sh: line 1: fg: no job control
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/base_serializers.py:28: UserWarning: A NumPy version >=1.23.5 and <2.3.0 is required for this version of SciPy (detected version 1.22.4)
  import scipy.sparse
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/python3/bin/sm-docker", line 8, in <module>
    sys.exit(main())
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker_studio_image_build/cli.py", line 133, in

In [11]:
!cat Dockerfile

#Download an open source TensorFlow Docker image
FROM ubuntu:18.04

RUN apt-get -y update && apt-get install -y --no-install-recommends \
         wget \
         python3-pip \
         python3-setuptools \
         nginx \
         ca-certificates \
    && rm -rf /var/lib/apt/lists/*

RUN ln -s /usr/bin/python3 /usr/bin/python
RUN ln -s /usr/bin/pip3 /usr/bin/pip
RUN pip install --upgrade pip
# Install sagemaker-training toolkit that contains the common functionality necessary to create a container compatible with SageMaker and the Python SDK.
#RUN pip install sagemaker-training
RUN pip --no-cache-dir install numpy==1.16.2 scipy==1.2.1 scikit-learn==0.20.2 pandas flask gunicorn

ENV PYTHONUNBUFFERED=TRUE
ENV PYTHONDONTWRITEBYTECODE=TRUE

# Copies the training code inside the container
COPY train.py /opt/ml/code/train.py

WORKDIR /opt/ml/code/

In [17]:
# Create a sagemaker session to upload data to S3
import sagemaker
sagemaker_session = sagemaker.Session()
# Upload data to default S3 bucket
prefix = "Heart_AWS"
training_input_path = sagemaker_session.upload_data("processed.cleveland.csv", key_prefix=prefix + "/training")

In [15]:
import os
print(os.listdir())


['build_and_push.sh', 'Heart_model.py', 'Heart_Docker.ipynb', 'gitignore', 'environment.yml', 'Dockerfile', 'build_and_push_studio.sh', '.git', 'data', '.ipynb_checkpoints', 'Heart_AWS.ipynb', 'containers']


In [22]:
import boto3

account_id = boto3.client('sts').get_caller_identity().get('Account')
ecr_repository = 'scikit-learn-custom'
tag = ':latest'

region = boto3.session.Session().region_name

uri_suffix = 'amazonaws.com'
if region in ['cn-north-1', 'cn-northwest-1']:
    uri_suffix = 'amazonaws.com.cn'

#byoc_image_uri = '{}.dkr.ecr.{}.{}/{}'.format(account_id, region, uri_suffix, ecr_repository + tag)
byoc_image_uri = '242201312017.dkr.ecr.us-east-2.amazonaws.com/scikit-learn-custom'

In [23]:
import sagemaker
from sagemaker import get_execution_role
from sagemaker.estimator import Estimator

estimator = Estimator(image_uri=byoc_image_uri,
                      role=get_execution_role(),
                      base_job_name='scikit-custom-container-test-job',
                      instance_count=1,
                      instance_type='ml.c5.xlarge')

# Train the estimator
estimator.fit({"train": training_input_path})

INFO:sagemaker:Creating training-job with name: scikit-custom-container-test-job-2024-09-01-22-34-00-505


ClientError: An error occurred (ValidationException) when calling the CreateTrainingJob operation: Cannot find the requested image: 242201312017.dkr.ecr.us-east-2.amazonaws.com/scikit-learn-custom with tag: latest and digest: null. Please check if your ECR image exists and role arn:aws:iam::242201312017:role/service-role/AmazonSageMaker-ExecutionRole-20240828T191393 has proper pull permissions for SageMaker: ecr:BatchCheckLayerAvailability, ecr:BatchGetImage, ecr:GetDownloadUrlForLayer